In [1]:
%matplotlib inline

import json
import matplotlib.pyplot as plt
import os
import io
import folium
import requests
import pandas as pd
from shapely.geometry import asShape, shape
from shapely.affinity import translate
from shapely.geometry.geo import mapping
from descartes import PolygonPatch

In [18]:
data = json.load(open("./data/countries.geojson"))
d_centroids = {}
for feat in data["features"]:
    geom = asShape(feat["geometry"])
    d_centroids[feat['id']] = [geom.centroid.x, geom.centroid.y]

In [19]:
d_centroids

{'AFG': [66.08669017344639, 33.85639924331851],
 'AGO': [17.470572472744326, -12.245868967981592],
 'ALB': [20.03242638605213, 41.14135336776871],
 'ARE': [54.20671462253365, 23.86863351514742],
 'ARG': [-65.17536065969124, -35.44682138709461],
 'ARM': [45.0002902364525, 40.21660746331794],
 'ATA': [30.02883672126619, -77.32786014887496],
 'ATF': [69.53158047042368, -49.30645491167198],
 'AUS': [134.50277546065465, -25.730654744883033],
 'AUT': [14.076158737088777, 47.61394873674072],
 'AZE': [47.55390962763196, 40.220690707965126],
 'BDI': [29.91390108115306, -3.377391780507063],
 'BEL': [4.580831170355562, 50.652442788532966],
 'BEN': [2.3373776582836743, 9.647430604325601],
 'BFA': [-1.7765373932154904, 12.311650411193026],
 'BGD': [90.26792836633709, 23.839461928066164],
 'BGR': [25.19511089646029, 42.753118773003436],
 'BHS': [-77.92997080393515, 25.515491725336627],
 'BIH': [17.816883262474374, 44.18076800389074],
 'BLR': [27.9813540194957, 53.5063443528151],
 'BLZ': [-88.7034212

In [24]:
d_names = {
    "ANTIOQUIA":'ANTIOQUIA',
    "ATLANTICO":'ATLANTICO',
    "SANTAFE DE BOGOTA D.C":'BOGOTA',
    "BOLIVAR":'BOLIVAR',
    "BOYACA":'BOYACA',
    "CALDAS":'CALDAS',
    "CAQUETA":'CAQUETA',
    "CAUCA":'CAUCA',
    "CESAR":'CESAR',
    "CORDOBA":'CORDOBA',
    "CUNDINAMARCA":'CUNDINAMARCA',
    "CHOCO":'CHOCO',
    "HUILA":'HUILA',
    "LA GUAJIRA":'GUAJIRA',
    "MAGDALENA":'MAGDALENA',
    "META":'META',
    "NARIÑO":'NARINO',
    "NORTE DE SANTANDER":'NSANTANDER',
    "QUINDIO":'QUINDIO',
    "RISARALDA":'RISARALDA',
    "SANTANDER":'SANTANDER',
    "SUCRE":'SUCRE',
    "TOLIMA":'TOLIMA',
    "VALLE DEL CAUCA":'VALLE',
    "ARAUCA":'ARAUCA',
    "CASANARE":'CASANARE',
    "PUTUMAYO":'PUTUMAYO',
    "AMAZONAS":'AMAZONAS',
    "GUAINIA":'GUAINIA',
    "GUAVIARE":'GUAVIARE',
    "VAUPES":'VAUPES',
    "VICHADA":'VICHADA',
    "ARCHIPIELAGO DE SAN ANDRES PROVIDENCIA Y SANTA CATALINA":'SANDRES'
}

In [26]:
r = requests.get('https://docs.google.com/spreadsheet/ccc?key=1e3PXAVDF_gPTx-_GTHU9hTAClEKJj5Y_D2_YsW6Vd6U&output=csv')
data = r.content
df_dptos = pd.read_csv(io.StringIO(data.decode("utf-8")))
df_dptos = df_dptos[['nombre_depto','area']]
df_dptos.head()

,nombre_depto,area
0,AMAZONAS,109665
1,ANTIOQUIA,63612
2,ARAUCA,23818
3,ATLANTICO,3388
4,BOGOTA,1587


In [29]:
df_countries = pd.read_csv('./data/areas.csv')
df_countries = df_countries[['Country Name','2017','Country Code']]
df_countries.index = df_countries['Country Name']
df_countries.head()

,Country Name,2017,Country Code
Country Name,,,
Aruba,Aruba,180.0,ABW
Afghanistan,Afghanistan,652860.0,AFG
Angola,Angola,1246700.0,AGO
Albania,Albania,28750.0,ALB
Andorra,Andorra,470.0,AND


In [30]:
def nearest_area(x):
    s = df_countries['2017'].apply(lambda y:abs(y-x))
    n_co = s[s==s.min()].index[0]
    return n_co

In [31]:
df_dptos['neartest_country'] = df_dptos['area'].apply(nearest_area)

In [32]:
df_dptos = pd.merge(left=df_dptos, right=df_countries, left_on='neartest_country', right_on='Country Name', how='inner')

/home/jfm/anaconda3/envs/colendatos/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3265: FutureWarning: 'Country Name' is both an index level and a column label.
Defaulting to column, but this will raise an ambiguity error in a future version
  exec(code_obj, self.user_global_ns, self.user_ns)


In [33]:
df_dptos['diff'] = df_dptos['area'] - df_dptos['2017']

In [34]:
df_dptos['p_diff'] = df_dptos['diff'].apply(abs)/df_dptos['area']

In [36]:
df_dptos.index = df_dptos['nombre_depto']

In [65]:
df_dptos.to_html()

'<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th></th>\n      <th>nombre_depto</th>\n      <th>area</th>\n      <th>neartest_country</th>\n      <th>Country Name</th>\n      <th>2017</th>\n      <th>Country Code</th>\n      <th>diff</th>\n      <th>p_diff</th>\n    </tr>\n    <tr>\n      <th>nombre_depto</th>\n      <th></th>\n      <th></th>\n      <th></th>\n      <th></th>\n      <th></th>\n      <th></th>\n      <th></th>\n      <th></th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <th>AMAZONAS</th>\n      <td>AMAZONAS</td>\n      <td>109665</td>\n      <td>Cuba</td>\n      <td>Cuba</td>\n      <td>109880.0</td>\n      <td>CUB</td>\n      <td>-215.0</td>\n      <td>0.001961</td>\n    </tr>\n    <tr>\n      <th>ANTIOQUIA</th>\n      <td>ANTIOQUIA</td>\n      <td>63612</td>\n      <td>Latvia</td>\n      <td>Latvia</td>\n      <td>64490.0</td>\n      <td>LVA</td>\n      <td>-878.0</td>\n      <td>0.013802</td>\n    </tr>\n    <tr>

In [11]:
import folium
import folium.plugins
from folium.features import *

In [56]:
data = json.load(open("./data/colombia.geojson"))
ava_deptos = []
for feat in data["features"]:
    new_features = []
    geom = feat["geometry"]
    if geom['type'] == "MultiPolygon":
        geom['coordinates'] = geom['coordinates'][-1]
        geom['type'] = "Polygon"
    geom = asShape(feat["geometry"])
    std_name = d_names[feat['properties']['NOMBRE_DPT']]
    nearest_co_code = df_dptos.loc[std_name]["Country Code"]
    #if feat['properties']['NOMBRE_DPT'] == "AMAZONAS":
    geom_d = geom
    if nearest_co_code in d_centroids:
        ava_deptos.append(std_name)
        x_co, y_co = d_centroids[nearest_co_code]
        x_de, y_de = geom_d.centroid.x, geom_d.centroid.y
        geom_d = asShape(feat["geometry"])
        geom_d = translate(geom_d, xoff=(x_co-x_de), yoff=(y_co-y_de), zoff=0.0)
        geom_d = mapping(geom_d)
        #print(geom_d)
        feat['geometry'] = geom_d
        new_features.append(feat)
        data["features"] = new_features
        with open('./data/translated_depto_%s.geojson' % std_name, 'w') as fp:
            json.dump(data, fp)
    
#data = json.load(open("./data/translated_depto.geojson"))

In [63]:
for depto in ava_deptos:

    colombia_map_0 = os.path.join('data', 'translated_depto_%s.geojson' % depto)

    m = folium.Map(
        location=[-59.1759, -11.6016],
        tiles='Mapbox Bright',
        zoom_start=2
    )

    def style_function(feature):
        return {'weight':'1', 'fillColor':'red', 'color':'red'}

    folium.GeoJson(
        colombia_map_0,
        name='geojson',
        style_function=style_function
    ).add_to(m)

    folium.LayerControl().add_to(m)

    m.save("./web/traslated_deptos/%s.html" % depto)